In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm.notebook import tqdm as tqdm


'''
def rk4(func, t, h, y, *x)
ルンゲ・クッタ法を一回分計算する関数
    引数リスト
    func:導関数
    t：現在時刻を表す変数
    h：刻み幅
    y：出力変数（求めたい値）
    *x:引数の数が可変する事に対応する、その他の必要変数
※この関数では時刻は更新されないため、これとは別に時間更新をする必要があります。
'''
def rk4(func, t, h, y, *x):
    k1=h*func(t, y, *x)
    k2=h*func(t+0.5*h, y+0.5*k1, *x)
    k3=h*func(t+0.5*h, y+0.5*k2, *x) 
    k4=h*func(t+h, y+k3, *x)
    y=y+(k1 + 2*k2 + 2*k3 + k4)/6
    return y

'''
導関数の書き方
def func(t, y, *state):
    func:自分で好きな関数名をつけられます
    t:時刻変数(変数の文字はtで無くても良い) 
    y:出力変数(変数の文字はyで無くても良い)
    *state:その他の必要変数(引数の数は可変可能))
#関数サンプル
def vdot(t, y, *state):
    s1=state[0]
    s2=state[1]
    return t+y+s1+s2
    
'''

def vdot(t, v, m ,f):
    return f/m

def xdot(t, x, v):
    return v



class Star:
    hoge=999
    G=6.67259e-11
    h=1.0
    def __init__(self, mass, pos ,velocity):
        self.mass=mass
        self.pos=np.array(pos)
        self.velocity=np.array(velocity)
    
    def move(self, member):
        sigmaF=np.zeros(2)
        for s in member:
            if s!=self:
                posvec=s.pos - self.pos
                vecnorm=np.linalg.norm(posvec, ord=2)
                unitvec=posvec/vecnorm
                f=unitvec*self.G*s.mass*self.mass/vecnorm**2
                sigmaF+=f
                
        vtemp=self.velocity
        postemp=self.pos
        self.velocity=rk4(vdot, 0, self.h, vtemp, self.mass, sigmaF)
        self.pos=rk4(xdot, 0, self.h, postemp, vtemp)
        
        
        
earth=Star(5.972e24,(0,0), (0,0))
moon=Star(7.348e22, (384400e3, -384400e3), (0, 1022))


member=(earth, moon)

X=[]
Y=[]
plt.figure(figsize=(8, 8))

counter=0

xm=moon.pos[0]
ym=moon.pos[1]

for i in tqdm(range(2390000)):
    moon.move(member)
    oldx=xm
    oldy=ym
    
    xm=moon.pos[0]
    ym=moon.pos[1]
    
    if (oldy<0 and ym>0):
        pass
        #break
    X.append(moon.pos[0])
    Y.append(moon.pos[1])
    
    if(i%10000==0):
    
        plt.plot(xm,ym,'o',color='green')
        plt.plot(0,0,'o',color='blue')
        plt.xlim(-160e7, 80e7)
        plt.ylim(-160e7, 80e7)
        plt.savefig('twobody_anim2/twobody{:04d}.png'.format(counter))
        plt.cla()
        counter+=1
    

In [ ]:
#GIF アニメにする
from PIL import Image
import glob

files = sorted(glob.glob('./twobody_anim2/*.png'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('./moon2.gif' , save_all = True , append_images = images[1:] , duration = 100 , loop = 0)